In [113]:
import pandas as pd
import os
import jieba
from langconv import * # convert Traditional Chinese characters to Simplified Chinese characters from https://github.com/Tony607/Chinese_sentiment_analysis

from os import listdir
from os.path import isfile, join

dataBaseDirPos = "../datasets/ChnSentiCorp_htl_ba_6000/train/pos/"
dataBaseDirNeg = "../datasets/ChnSentiCorp_htl_ba_6000/train/neg/"
positiveFiles = [dataBaseDirPos + f for f in listdir(dataBaseDirPos) if isfile(join(dataBaseDirPos, f))]
negativeFiles = [dataBaseDirNeg + f for f in listdir(dataBaseDirNeg) if isfile(join(dataBaseDirNeg, f))]

In [114]:
print(len(positiveFiles))
print(len(negativeFiles))

2916
3000


In [115]:
filename = positiveFiles[0]
with codecs.open(filename, "rb") as doc_file:
    text=doc_file.read().decode("GB2312")
    print(text)

In [116]:
train_data = []
for filename in positiveFiles: # Preprocessing (like the owner of the github repository does)
    text=""
    with codecs.open(filename, "rb") as doc_file:
        for line in doc_file:
            try:
                line = line.decode("GB2312")
            except:
                continue
            text+=Converter('zh-hans').convert(line)# Convert from traditional to simplified Chinese

            text = text.replace("\n", "")
            text = text.replace("\r", "")
    train_data.append((text, "pos"))

n_pos = len(train_data)
print(str(n_pos))

for filename in negativeFiles: # Preprocessing (like the owner of the github repository does)
    text=""
    with codecs.open(filename, "rb") as doc_file:
        for line in doc_file:
            try:
                line = line.decode("GB2312")
            except:
                continue
            text+=Converter('zh-hans').convert(line)# Convert from traditional to simplified Chinese

            text = text.replace("\n", "")
            text = text.replace("\r", "")
    train_data.append((text, "neg"))

n_neg = len(train_data) - n_pos
print(str(n_neg))

2916
3000


In [117]:
import random

random.shuffle(train_data)

In [118]:
# Cada review es una lista de tokens, la salida será "pos" o "neg". Las stopwords las dejamos porque el dataset es muy pequenio (no ahorra mucho tiempo de entrenamiento).
# Tokenize only
X = []
y = [str(data[1]) for data in train_data]
for data in train_data:
    seg_list = jieba.cut(data[0], cut_all=False)
    seg_list = list(seg_list)
    X.append(seg_list)
    
X[:3]

[['第一次',
  '住金',
  '海棠',
  '，',
  '完全',
  '没有',
  '想到',
  '在',
  '市区',
  '里会',
  '有',
  '这样',
  '花园式',
  '的',
  '酒店',
  '，',
  '处处',
  '绿萌',
  '盈盈',
  '。',
  '5',
  '号楼',
  '在',
  '最',
  '里面',
  '，',
  '应该',
  '是',
  '房间',
  '最好',
  '的',
  '。',
  '非常',
  '安静',
  '，',
  '房间',
  '窗外',
  '就是',
  '山林',
  '，',
  '空气',
  '很',
  '清新',
  '。',
  '进屋',
  '时',
  '发现',
  '窗子',
  '已经',
  '打开',
  '通风',
  '，',
  '所以',
  '屋内',
  '的',
  '空气',
  '是',
  '一般',
  '酒店',
  '里',
  '难得',
  '的',
  '清新',
  '。',
  '地理位置',
  '离',
  '乐山',
  '在',
  '佛',
  '也',
  '不远',
  '。',
  '不足之处',
  '是',
  '没有',
  '健身房',
  '。',
  '除此以外',
  '，',
  '金',
  '海棠',
  '都',
  '很',
  '好',
  '，',
  '强烈推荐',
  '！'],
 ['还',
  '不错',
  '，',
  '应该',
  '比较',
  '符合',
  '三星级',
  '标准',
  '，',
  '不算',
  '太贵',
  '，',
  '可能',
  '是',
  '和',
  '携程',
  '的',
  '签约',
  '价格',
  '！'],
 ['查看',
  '了',
  '很多',
  '的',
  '游记',
  '，',
  '选择',
  '了',
  '这个',
  '酒店',
  '。',
  '很',
  '失望',
  '，',
  '建议',
  '以后',
  '大家',
  '不要',
  '住',
  '。',
  '到',
  '前台',
  '，',
  '

In [119]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

h = sorted([len(sentence) for sentence in X])
maxLength = h[int(len(h) * 0.60)]
print("Max length is: ",h[len(h)-1])
print("60% cover length up to: ",maxLength)

X = [" ".join(wordslist) for wordslist in X]  # Keras Tokenizer expect the words tokens to be seperated by space 

input_tokenizer = Tokenizer(30000) # Initial vocab size
input_tokenizer.fit_on_texts(X)

vocab_size = len(input_tokenizer.word_index) + 1
print("input vocab_size:", vocab_size)

X = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X), maxlen=maxLength))

Max length is:  1804
60% cover length up to:  68
input vocab_size: 22123


In [120]:
target_tokenizer = Tokenizer(3)
target_tokenizer.fit_on_texts(y)

print("output vocab_size:", len(target_tokenizer.word_index) + 1)

y = np.array(target_tokenizer.texts_to_sequences(y)) -1
y = y.reshape(y.shape[0])

output vocab_size: 3


In [121]:
len(X)

5916

In [122]:
len(y)

5916

In [123]:
X[:3]

array([[    3,    88,   840,    11,   170,   237,     1,   190,     6,
            8,   349,     2,     3,    42,   324,     1,     8,   763,
           37,  7024,     1,   962,     9,  5760,     3,  3237,    71,
          133,  2222,   127,   431,  1300,     1,   144,  2530,     2,
          962,     6,    70,     5,    91,  2310,     2,  5760,     3,
          229,   147,  4070,    11, 13627,    12,   744,     3,  2138,
            6,    16,  1637,     3, 13628,     1,  3453,  8707,    15,
            9,    23,     1,  2531,    10],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,    17,    22,     1,   190,    33,   891,
          702,   171,     1,   62

In [124]:
# Number of features
n_cols=len(X[0])
n_cols

68

In [125]:
col_names=[str(i) for i in range(n_cols)]

df = pd.DataFrame(X, columns=col_names) 
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,3,88,840,11,170,237,1,190,6,8,...,13628,1,3453,8707,15,9,23,1,2531,10
1,0,0,0,0,0,0,0,0,0,0,...,971,1,192,6,25,40,2,2532,53,10
2,390,1,15,34,211,3454,3,2421,20,13631,...,214,422,984,266,1,173,20,54,5,3


In [126]:
y[:3]

array([1, 1, 0])

In [127]:
df['class']=y
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,class
0,3,88,840,11,170,237,1,190,6,8,...,1,3453,8707,15,9,23,1,2531,10,1
1,0,0,0,0,0,0,0,0,0,0,...,1,192,6,25,40,2,2532,53,10,1
2,390,1,15,34,211,3454,3,2421,20,13631,...,422,984,266,1,173,20,54,5,3,0


In [128]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15)

In [129]:
from auto_ml import Predictor

column_descriptions = {
  'class': 'output',
}

ml_predictor = Predictor(type_of_estimator='classifier', column_descriptions=column_descriptions)

In [130]:
ml_predictor.train(train)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.10
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Fitting DataFrameVectorizer


C:\Users\berna\Anaconda3\envs\tfginfo\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingClassifier to predict class
Started at:
2020-03-27 03:00:11
[1] random_holdout_set_from_training_data's score is: -0.247
[2] random_holdout_set_from_training_data's score is: -0.245


C:\Users\berna\Anaconda3\envs\tfginfo\lib\site-packages\sklearn\ensemble\_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)
C:\Users\berna\Anaconda3\envs\tfginfo\lib\site-packages\sklearn\ensemble\_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)
C:\Users\berna\Anaconda3\envs\tfginfo\lib\site-packages\sklearn\ensemble\_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradie

[3] random_holdout_set_from_training_data's score is: -0.243
[4] random_holdout_set_from_training_data's score is: -0.242
[5] random_holdout_set_from_training_data's score is: -0.241
[6] random_holdout_set_from_training_data's score is: -0.24
[7] random_holdout_set_from_training_data's score is: -0.239
[8] random_holdout_set_from_training_data's score is: -0.238


...

C:\Users\berna\Anaconda3\envs\tfginfo\lib\site-packages\sklearn\ensemble\_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


[1580] random_holdout_set_from_training_data's score is: -0.185


C:\Users\berna\Anaconda3\envs\tfginfo\lib\site-packages\sklearn\ensemble\_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


[1600] random_holdout_set_from_training_data's score is: -0.185
The number of estimators that were the best for this training dataset: 1200
The best score on the holdout set: -0.18241115034449268
Finished training the pipeline!
Total training time:
0:01:11


Here are the results from our GradientBoostingClassifier
predicting class
Calculating feature responses, for advanced analytics.


C:\Users\berna\Anaconda3\envs\tfginfo\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [131]:
# Score the model on test data
test_score = ml_predictor.score(test, test['class'])

Here is our brier-score-loss, which is the default value we optimized for while training, and is the value returned from .score() unless you requested a custom scoring metric
It is a measure of how close the PROBABILITY predictions are.
0.1768

Here is the trained estimator's overall accuracy (when it predicts a label, how frequently is that the correct label?)
74.0%

Here is a confusion matrix showing predictions vs. actuals by label:
Predicted >    0    1  All
v Actual v                
0            349   95  444
1            136  308  444
All          485  403  888

Here is predictive value by class:
Class:  0 = 0.7195876288659794
Class:  1 = 0.7642679900744417
+------------------+-----------------------------------+--------------------------------+
| Bucket Edges     |   Predicted Probability Of Bucket |   Actual Probability of Bucket |
|------------------+-----------------------------------+--------------------------------|
| (0.0005, 0.0655] |                            0.0398 | 

In [132]:
test_score

-0.17682401838029216

In [133]:
from auto_ml.utils_models import load_ml_model

file_name = ml_predictor.save()

# trained_model = load_ml_model(file_name)



We have saved the trained pipeline to a filed called "auto_ml_saved_pipeline.dill"
It is saved in the directory: 
C:\Users\berna\Documents\kaggle_notebooks
To use it to get predictions, please follow the following flow (adjusting for your own uses as necessary:


`from auto_ml.utils_models import load_ml_model
`trained_ml_pipeline = load_ml_model("auto_ml_saved_pipeline.dill")
`trained_ml_pipeline.predict(data)`


Note that this pickle/dill file can only be loaded in an environment with the same modules installed, and running the same Python version.
This version of Python is:
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)


When passing in new data to get predictions on, columns that were not present (or were not found to be useful) in the training data will be silently ignored.
It is worthwhile to make sure that you feed in all the most useful data points though, to make sure you can get the highest quality predictions.
